In [1]:
import pandas as pd
import numpy as np
import os, yaml, wandb, pickle, optuna, gc

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import TensorDataset, DataLoader, Sampler

from rl import make_transition, make_transition_for_AKI, imvt, CustomSampler, CustomSampler2, imvt2,cql_loss
from make_plot import show_AUROC, plot_alpha, plot_beta, make_transition_test, make_betas

from tqdm import tqdm
from sklearn.metrics import roc_curve, roc_auc_score

if os.getcwd()[-4:] == "code":
    os.chdir('../')
    
with open(os.path.join("./code/params.yaml")) as f:
        params = yaml.safe_load(f)
    
np.random.seed(params['random_seed'])
torch.manual_seed(params['random_seed'])
if torch.cuda.is_available():
    torch.cuda.manual_seed(params['random_seed'])
    torch.cuda.manual_seed_all(params['random_seed'])

c:\Users\CYP\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = './code/train_4hrs_mean.parquet'
val = './code/val_4hrs_mean.parquet'

train_data = make_transition(train,rolling_size=6)
val_data = make_transition(val,rolling_size=6)

100%|██████████| 4390/4390 [00:04<00:00, 1035.75it/s]


In [3]:
val_transition = make_transition_test(val,rolling_size=6)

100%|██████████| 4390/4390 [00:04<00:00, 945.15it/s] 


In [4]:
def objective(trial):
    batch_size = trial.suggest_categorical("batch_size",[32,64,128,256])
    n_units = trial.suggest_int("n_units",1,64)
    
    lr = trial.suggest_categorical("learning_rate",[1e-6,5e-6,1e-5,5e-5,1e-4,5e-4, 1e-3, 5e-3, 1e-2])
    lr_decay = trial.suggest_float("lr_decay",0.5,0.999)
    lr_step = trial.suggest_int("lr_step",1,20)

    ns = trial.suggest_int("negative_sampling",2,16)
    
    loss = trial.suggest_categorical("loss",['smooth_l1','mse'])
    CQL = trial.suggest_categorical("CQL",[0])

    update_freq = trial.suggest_int("update_freq",2,128)
    
    PPO = trial.suggest_categorical("PPO",[0])
    
    epochs = 50

    wandb.init(
        project='IMV_LSTM_Rescue_new', name=f'E-{trial.number+1}', reinit=True,
        config={
        "batch_size":batch_size,
        "n_units":n_units,
        "learning_rate":lr,
        "lr_decay":lr_decay,
        "lr_step":lr_step,
        "ns":ns,
        "loss":loss,
        "update_freq":update_freq,
        "PPO":PPO,
        "CQL":CQL
    })

    auroc = train(batch_size,n_units,lr,lr_decay,lr_step,ns,loss,epochs,update_freq,PPO,CQL)

    return auroc

In [5]:
def train(batch_size,n_units,lr,lr_decay,lr_step,ns,loss_type,epochs,update_freq,PPO,CQL):
    network = imvt(input_dim=params['state_dim'], output_dim=params['num_actions'], n_units=n_units, device=device).to(device)
    target_network = imvt(input_dim=params['state_dim'], output_dim=params['num_actions'], n_units=n_units, device=device).to(device)
    gamma = 1.0

    optimizer = optim.Adam(network.parameters(), lr=lr)
    scheduler = ExponentialLR(optimizer, gamma=lr_decay)

    num_workers = 4

    patience = 5
    best_loss = 1e6
    counters = 0

    sampler = CustomSampler2(train_data,batch_size,ns=ns)
    train_loader = DataLoader(train_data,batch_sampler=sampler,num_workers=num_workers)
    val_loader = DataLoader(val_data,batch_size=256,shuffle=False)

    for epoch in range(epochs):
        train_loss = 0
        update_counter = 0
        for s,a,r1,r2,r3,s2,t in tqdm(train_loader):
            s = s.to(device)
            a = a.to(device)
            r = r2.to(device)
            s2 = s2.to(device)
            t = t.to(device)

            q,_,_ = network(s)
            q_pred = q.gather(1, a).squeeze()
            
            with torch.no_grad():
                q2,_,_ = target_network(s2)
                q2_net,_, _ = network(s2)

            q2_max = q2.gather(1, torch.max(q2_net,dim=1)[1].unsqueeze(1)).squeeze(1).detach()

            bellman_target = r + gamma * q2_max.detach()*(1-t)
            if loss_type == "l1":loss = F.l1_loss(q_pred, bellman_target)
            elif loss_type == "smooth_l1":loss = F.smooth_l1_loss(q_pred, bellman_target)
            elif loss_type == "mse":loss = F.mse_loss(q_pred, bellman_target)
            
            if CQL:
                loss += cql_loss(q,a)

            optimizer.zero_grad()
            loss.backward()
            if PPO:
                for param in network.parameters():
                    param.grad.data.clamp_(-1, 1)
            train_loss += loss.item()
            optimizer.step()

            update_counter += 1
            if update_counter == update_freq:
                target_network.load_state_dict(network.state_dict())
                update_counter = 0

        with torch.no_grad():
            val_loss = 0
            for s,a,r1,r2,r3,s2,t in val_loader:
                s = s.to(device)
                a = a.to(device)
                r = r2.to(device)
                s2 = s2.to(device)
                t = t.to(device)

                q,_,_ = network(s)
                q2,_,_ = target_network(s2)
                q2 = q2.detach()
                q_pred = q.gather(1, a).squeeze()

                q2_net,_,_ = network(s2)
                q2_net = q2_net.detach()
                q2_max = q2.gather(1, torch.max(q2_net,dim=1)[1].unsqueeze(1)).squeeze()

                bellman_target = r + gamma * q2_max.detach()*(1-t)
                if loss_type == "l1":loss = F.l1_loss(q_pred, bellman_target)
                elif loss_type == "smooth_l1":loss = F.smooth_l1_loss(q_pred, bellman_target)
                elif loss_type == "mse":loss = F.mse_loss(q_pred, bellman_target)
                if CQL:
                    loss += cql_loss(q,a)
                val_loss += loss.item()

            q_value = []
            reward = []
            rescue = []
            rescue_24 = []
            for s,a,r1,r2,r3,r1p,r2p,r24 in val_transition:
                s = s.to(device)
                q,_,_ = network(s.to(device))
                q_value.append(q.detach().cpu().numpy())
                reward.append(r2.detach().cpu().numpy())
                rescue.append(r2p.detach().cpu().numpy())
                rescue_24.append(r24.detach().cpu().numpy())
            q_value = np.concatenate(q_value,axis=0)
            reward = np.concatenate(reward,axis=0)
            rescue = np.concatenate(rescue,axis=0)
            rescue_24 = np.concatenate(rescue_24,axis=0)
            q_max = q_value.max(axis=1)
            q_median = np.median(q_value, axis=1)
            auroc = roc_auc_score(reward,q_max)
            auroc_med = roc_auc_score(reward,q_median)
            auroc_p = roc_auc_score(rescue,q_max)
            auroc_p_med = roc_auc_score(rescue,q_median)
            auroc24 = roc_auc_score(rescue_24,q_max)
            auroc24_med = roc_auc_score(rescue_24,q_median)
            
        
        
        if (epoch%lr_step ==0):
            scheduler.step()
                
        if val_loss < best_loss:
            best_loss = val_loss
            counters = 0
        else :
            counters += 1
            
        wandb.log({"Iter:": epoch, "train:":train_loss, "val:":val_loss, "AUROC":auroc, "AUROC_median":auroc_med,"auroc_p":auroc_p,"auroc_p_med":auroc_p_med,"auroc_24":auroc24,"auroc_24_med":auroc24_med})


        
        if (counters > patience)&(epoch>=20):
            break

    gc.collect()
    torch.cuda.empty_cache()
    return auroc

In [6]:
device = 'cuda:0'
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)

best_params = study.best_params
best_loss = study.best_value

print("Best Hyperparameters:", best_params)
print("Best Validation Loss:", best_loss)

[I 2024-03-19 01:00:59,168] A new study created in memory with name: no-name-a33870f8-a442-458e-b7aa-4299acbb58bc
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chanreverse (dahs). Use `wandb login --relogin` to force relogin


 99%|█████████▉| 5112/5145 [01:00<00:00, 84.73it/s] 
[I 2024-03-19 01:25:04,552] Trial 0 finished with value: 0.3194561164518299 and parameters: {'batch_size': 128, 'n_units': 3, 'learning_rate': 0.0005, 'lr_decay': 0.5177803359555104, 'lr_step': 13, 'negative_sampling': 5, 'loss': 'mse', 'CQL': 0, 'update_freq': 118, 'PPO': 0}. Best is trial 0 with value: 0.3194561164518299.


AUROC,█▅▄▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
AUROC_median,█▅▄▄▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
Iter:,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
auroc_24,█▆▅▄▄▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁
auroc_24_med,█▆▅▄▄▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁
auroc_p,▇██▇▇▆▆▅▄▃▃▃▃▂▂▂▂▂▂▁▁
auroc_p_med,▇██▇▇▇▆▅▄▃▃▃▂▂▂▂▂▁▂▁▁
train:,█▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂
val:,▁▁▁▁▁▂▂▂▂▂▃▃▄▄▅▅▆▆▇██
AUROC,0.31946
AUROC_median,0.32596


2598it [00:39, 65.40it/s]                          
2598it [00:35, 73.06it/s]                          
2598it [00:34, 75.50it/s]                          
2598it [00:34, 76.03it/s]                          
2598it [00:35, 74.20it/s]                          
2598it [00:33, 77.91it/s]                          
2598it [00:34, 75.38it/s]                          
2598it [00:34, 75.96it/s]                          
2598it [00:34, 74.34it/s]                          
2598it [00:34, 74.65it/s]                          
2598it [00:35, 72.83it/s]                          
2598it [00:34, 74.77it/s]                          
2598it [00:34, 75.59it/s]                          
2598it [00:34, 75.45it/s]                          
2598it [00:33, 77.90it/s]                          
2598it [00:33, 77.92it/s]                          
2598it [00:34, 74.84it/s]                          
2598it [00:34, 75.15it/s]                          
2598it [00:35, 72.53it/s]                          
2598it [00:3

AUROC,██▇▅▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
AUROC_median,██▇▄▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
Iter:,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
auroc_24,██▇▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
auroc_24_med,██▇▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
auroc_p,▄▄▃▁█▆▄▄▄▅▅▅▅▆▅▆▆▆▆▆▆
auroc_p_med,▅▄▃▁█▆▄▄▄▅▅▅▅▆▅▆▆▆▇▇▇
train:,█▆▅▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val:,▅█▅▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
AUROC,0.31549
AUROC_median,0.31223


...

 97%|█████████▋| 2495/2572 [00:38<00:01, 64.05it/s]
[I 2024-03-19 11:08:47,656] Trial 15 finished with value: 0.3648690971312132 and parameters: {'batch_size': 256, 'n_units': 29, 'learning_rate': 0.01, 'lr_decay': 0.9171296050521924, 'lr_step': 7, 'negative_sampling': 4, 'loss': 'mse', 'CQL': 0, 'update_freq': 5, 'PPO': 0}. Best is trial 14 with value: 0.6671807459480341.


AUROC,▁▃▅▂▄▅▅▄▅▇▅▅▅▆▆▇▇▆█▇▇
AUROC_median,▁▂▄▃▄▄▄▄▅▇▅▅▅▆▆▇▇▆█▆▇
Iter:,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
auroc_24,▁▃▄▂▃▄▄▄▅▇▅▅▅▆▆▇▇▆█▇▇
auroc_24_med,▁▁▃▂▃▃▄▃▅▇▅▅▅▆▆▇▇▆█▇▇
auroc_p,█▄▂▅▃▁▁▂▅▅▃▅▄▆▄▅▄▅▆▄▄
auroc_p_med,█▄▂▅▃▁▁▂▅▅▃▅▄▆▅▅▄▅▆▄▄
train:,▁▄▄▄▃▃▄▄▄▅▅▆▆▆▆▆▆▇█▆▅
val:,▂▃▃▁▁▁▂▂▃▄▄▅▅▆▄▆▅█▆▅▃
AUROC,0.36487
AUROC_median,0.36389


 97%|█████████▋| 2495/2572 [00:33<00:01, 73.56it/s]
[I 2024-03-19 11:23:00,575] Trial 16 finished with value: 0.39289303795286246 and parameters: {'batch_size': 256, 'n_units': 13, 'learning_rate': 0.01, 'lr_decay': 0.750629824072466, 'lr_step': 14, 'negative_sampling': 4, 'loss': 'mse', 'CQL': 0, 'update_freq': 63, 'PPO': 0}. Best is trial 14 with value: 0.6671807459480341.


AUROC,█▇▇▆▆▆▅▄▆▆▆▅▅▄▅▅▅▃▄▂▁
AUROC_median,█▇▇▆▆▆▆▅▆▆▆▅▅▄▆▅▅▃▄▃▁
Iter:,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
auroc_24,█▇▇▆▆▆▅▄▆▆▆▅▅▄▅▅▅▃▄▂▁
auroc_24_med,█▇▇▆▆▆▆▄▆▆▆▆▅▄▅▅▅▃▄▃▁
auroc_p,████▇█▇▆▇▇▇▇▆▅▇▆▆▄▄▂▁
auroc_p_med,████▇█▇▆▇▇▇▇▆▅▇▆▆▄▃▂▁
train:,█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val:,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
AUROC,0.39289
AUROC_median,0.3794


26202it [04:54, 88.84it/s]                            
26202it [04:49, 90.61it/s]                            
26202it [04:51, 89.79it/s]                            
26202it [04:48, 90.70it/s]                            
26202it [04:51, 89.95it/s]                            
26202it [04:53, 89.27it/s]                            
26202it [04:56, 88.52it/s]                            
26202it [04:49, 90.42it/s]                            
26202it [04:53, 89.38it/s]                            
 97%|█████████▋| 19937/20583 [03:41<00:06, 95.04it/s] 